<a href="https://colab.research.google.com/github/xmrcl0/SME0829/blob/main/project2/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 2 - Car Price Prediction

In [ ]:
import warnings
warnings.filterwarnings('ignore')
!pip install category_encoders
!pip install catboost

# Introdução
Neste projeto, vamos construir um modelo capaz de sugerir o preço de venda de carros usados, a partir das características do veículo (marca e modelo, ano de fabricação, quilometros rodados, consumo de combustível, etc.). 

# Objetivo
Construir um modelo estimador de preços de venda para carros usados, a partir de um dataset fornecido.

# Procedimento
1. Análise exploratória dos dados
2. Pré-processamento
3. Feature engineering
4. Modelagem
  1. Experimento 1
  2. Experimento 2
  3. Experimento 3
  4. Experimento 4
5. Resultado
6. Validação
  1. Pré-processamento
  2. Feature engineering
  3. Predição
7. Conclusão

## Análise exploratória dos dados
Análises detalhadas foram fornecidas (publicar no github pages).

In [ ]:
# Load train dataset
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/xmrcl0/SME0829/main/project2/data/train_car_details.csv')

In [ ]:
# Show dataset head
df.head()

,Id,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,1,Hyundai Santro GLS I - Euro I,1999,80000,110000,Petrol,Individual,Manual,Second Owner,NaN,NaN,NaN,NaN,NaN
1,2,Maruti Ertiga VDI,2012,459999,87000,Diesel,Individual,Manual,First Owner,20.77 kmpl,1248 CC,88.76 bhp,200Nm@ 1750rpm,7.0
2,3,BMW 3 Series 320d Luxury Line,2010,1100000,102000,Diesel,Dealer,Automatic,First Owner,19.62 kmpl,1995 CC,187.74 bhp,400Nm@ 1750-2500rpm,5.0
3,4,Tata New Safari DICOR 2.2 EX 4x2,2009,229999,212000,Diesel,Individual,Manual,Third Owner,11.57 kmpl,2179 CC,138.1 bhp,320Nm@ 1700-2700rpm,7.0
4,5,Toyota Fortuner 3.0 Diesel,2010,800000,125000,Diesel,Individual,Manual,Second Owner,11.5 kmpl,2982 CC,171 bhp,343Nm@ 1400-3400rpm,7.0


In [ ]:
# Show data information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5689 entries, 0 to 5688
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             5689 non-null   int64  
 1   name           5689 non-null   object 
 2   year           5689 non-null   int64  
 3   selling_price  5689 non-null   int64  
 4   km_driven      5689 non-null   int64  
 5   fuel           5689 non-null   object 
 6   seller_type    5689 non-null   object 
 7   transmission   5689 non-null   object 
 8   owner          5689 non-null   object 
 9   mileage        5532 non-null   object 
 10  engine         5532 non-null   object 
 11  max_power      5538 non-null   object 
 12  torque         5531 non-null   object 
 13  seats          5532 non-null   float64
dtypes: float64(1), int64(4), object(9)
memory usage: 622.4+ KB


In [ ]:
# Show simple statistics (numerical columns)
df.describe()

,Id,year,selling_price,km_driven,seats
count,5689.000000,5689.000000,5.689000e+03,5.689000e+03,5532.000000
mean,2845.000000,2013.741607,6.277744e+05,6.987759e+04,5.422993
std,1642.417172,4.054875,7.784000e+05,5.124731e+04,0.961395
min,1.000000,1983.000000,2.999900e+04,1.000000e+00,2.000000
25%,1423.000000,2011.000000,2.500000e+05,3.500000e+04,5.000000
50%,2845.000000,2014.000000,4.500000e+05,6.000000e+04,5.000000
75%,4267.000000,2017.000000,6.670000e+05,9.800000e+04,5.000000
max,5689.000000,2020.000000,7.200000e+06,1.500000e+06,10.000000


In [ ]:
# Search for duplicated lines
df = df.drop(labels='Id', axis=1)
len(df[df.duplicated()]) 

700

## Pré-processamento
Durante a análise exploratória, foi verificado que a coluna torque possui valores com diferentes unidades (Nm e Kgm). Devido a isso, por simplicidade, a coluna não será usada. Dependendo do resultado obtido, vale a pena avaliar seu uso em um trabalho futuro.

In [ ]:
# Show the training dataset
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Hyundai Santro GLS I - Euro I,1999,80000,110000,Petrol,Individual,Manual,Second Owner,NaN,NaN,NaN,NaN,NaN
1,Maruti Ertiga VDI,2012,459999,87000,Diesel,Individual,Manual,First Owner,20.77 kmpl,1248 CC,88.76 bhp,200Nm@ 1750rpm,7.0
2,BMW 3 Series 320d Luxury Line,2010,1100000,102000,Diesel,Dealer,Automatic,First Owner,19.62 kmpl,1995 CC,187.74 bhp,400Nm@ 1750-2500rpm,5.0
3,Tata New Safari DICOR 2.2 EX 4x2,2009,229999,212000,Diesel,Individual,Manual,Third Owner,11.57 kmpl,2179 CC,138.1 bhp,320Nm@ 1700-2700rpm,7.0
4,Toyota Fortuner 3.0 Diesel,2010,800000,125000,Diesel,Individual,Manual,Second Owner,11.5 kmpl,2982 CC,171 bhp,343Nm@ 1400-3400rpm,7.0


In [ ]:
# Remove duplicated lines. (Un)comment this line to see the effect in R2 score
#df = df.drop_duplicates()
len(df[df.duplicated()]) 

700

In [ ]:
# Remove unit from the max_power feature
df = df[df['max_power'].notna()]
df['max_power'] = df['max_power'].apply(lambda power: pd.Series(power.split()[0]))
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
1,Maruti Ertiga VDI,2012,459999,87000,Diesel,Individual,Manual,First Owner,20.77 kmpl,1248 CC,88.76,200Nm@ 1750rpm,7.0
2,BMW 3 Series 320d Luxury Line,2010,1100000,102000,Diesel,Dealer,Automatic,First Owner,19.62 kmpl,1995 CC,187.74,400Nm@ 1750-2500rpm,5.0
3,Tata New Safari DICOR 2.2 EX 4x2,2009,229999,212000,Diesel,Individual,Manual,Third Owner,11.57 kmpl,2179 CC,138.1,320Nm@ 1700-2700rpm,7.0
4,Toyota Fortuner 3.0 Diesel,2010,800000,125000,Diesel,Individual,Manual,Second Owner,11.5 kmpl,2982 CC,171,343Nm@ 1400-3400rpm,7.0
5,Maruti Alto LX,2010,180000,25000,Petrol,Individual,Manual,Second Owner,19.7 kmpl,796 CC,46.3,62Nm@ 3000rpm,5.0


In [ ]:
# Remove unit from the engine feature
df = df[df['engine'].notna()]
df['engine'] = df['engine'].apply(lambda power: pd.Series(power.split()[0]))
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
1,Maruti Ertiga VDI,2012,459999,87000,Diesel,Individual,Manual,First Owner,20.77 kmpl,1248,88.76,200Nm@ 1750rpm,7.0
2,BMW 3 Series 320d Luxury Line,2010,1100000,102000,Diesel,Dealer,Automatic,First Owner,19.62 kmpl,1995,187.74,400Nm@ 1750-2500rpm,5.0
3,Tata New Safari DICOR 2.2 EX 4x2,2009,229999,212000,Diesel,Individual,Manual,Third Owner,11.57 kmpl,2179,138.1,320Nm@ 1700-2700rpm,7.0
4,Toyota Fortuner 3.0 Diesel,2010,800000,125000,Diesel,Individual,Manual,Second Owner,11.5 kmpl,2982,171,343Nm@ 1400-3400rpm,7.0
5,Maruti Alto LX,2010,180000,25000,Petrol,Individual,Manual,Second Owner,19.7 kmpl,796,46.3,62Nm@ 3000rpm,5.0


In [ ]:
# Remove unit from the mileage feature
df = df[df['mileage'].notna()]
df['mileage'] = df['mileage'].apply(lambda power: pd.Series(power.split()[0]))
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
1,Maruti Ertiga VDI,2012,459999,87000,Diesel,Individual,Manual,First Owner,20.77,1248,88.76,200Nm@ 1750rpm,7.0
2,BMW 3 Series 320d Luxury Line,2010,1100000,102000,Diesel,Dealer,Automatic,First Owner,19.62,1995,187.74,400Nm@ 1750-2500rpm,5.0
3,Tata New Safari DICOR 2.2 EX 4x2,2009,229999,212000,Diesel,Individual,Manual,Third Owner,11.57,2179,138.1,320Nm@ 1700-2700rpm,7.0
4,Toyota Fortuner 3.0 Diesel,2010,800000,125000,Diesel,Individual,Manual,Second Owner,11.5,2982,171,343Nm@ 1400-3400rpm,7.0
5,Maruti Alto LX,2010,180000,25000,Petrol,Individual,Manual,Second Owner,19.7,796,46.3,62Nm@ 3000rpm,5.0


In [ ]:
# Remove rows with empty torque and after the entire column (it helps, believe)
df = df[df['torque'].notna()]
df = df.drop(labels='torque', axis=1)
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats
1,Maruti Ertiga VDI,2012,459999,87000,Diesel,Individual,Manual,First Owner,20.77,1248,88.76,7.0
2,BMW 3 Series 320d Luxury Line,2010,1100000,102000,Diesel,Dealer,Automatic,First Owner,19.62,1995,187.74,5.0
3,Tata New Safari DICOR 2.2 EX 4x2,2009,229999,212000,Diesel,Individual,Manual,Third Owner,11.57,2179,138.1,7.0
4,Toyota Fortuner 3.0 Diesel,2010,800000,125000,Diesel,Individual,Manual,Second Owner,11.5,2982,171,7.0
5,Maruti Alto LX,2010,180000,25000,Petrol,Individual,Manual,Second Owner,19.7,796,46.3,5.0


## Featuring Engineering

In [ ]:
# Replace car's name column by the car's brand
df['name'] = df['name'].apply(lambda name: pd.Series(name.split()[0]))
df.rename(columns={'name':'brand'}, inplace=True)
df.head()

,brand,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats
1,Maruti,2012,459999,87000,Diesel,Individual,Manual,First Owner,20.77,1248,88.76,7.0
2,BMW,2010,1100000,102000,Diesel,Dealer,Automatic,First Owner,19.62,1995,187.74,5.0
3,Tata,2009,229999,212000,Diesel,Individual,Manual,Third Owner,11.57,2179,138.1,7.0
4,Toyota,2010,800000,125000,Diesel,Individual,Manual,Second Owner,11.5,2982,171,7.0
5,Maruti,2010,180000,25000,Petrol,Individual,Manual,Second Owner,19.7,796,46.3,5.0


In [ ]:
# Save one copy of the enriched dataframe
df1 = df

In [ ]:
len(df[df.duplicated()]) 

704

In [ ]:
# Remove duplicated lines. (Un)comment this line to see the effect in R2 score
df = df.drop_duplicates()
len(df[df.duplicated()]) 

0

## Modelagem


### Experimento 1
* Objetivo: Verificar a performance do modelo, usando como parâmetros LabelEncoding, StandardScaler e CatboostRegressor.
* Features: brand, fuel, seller_type, owner, seats
* Encoder: LabelEncoding
* Scaler: StandardScaler
* Algoritm: CatBoostRegressor

In [ ]:
# Create features dataframe
features = ['brand', 'year', 'km_driven', 'fuel', 'seller_type', 'transmission', 'owner', 'mileage', 'engine', 'max_power', 'seats']
X = df[features]
X.head()

,brand,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats
1,Maruti,2012,87000,Diesel,Individual,Manual,First Owner,20.77,1248,88.76,7.0
2,BMW,2010,102000,Diesel,Dealer,Automatic,First Owner,19.62,1995,187.74,5.0
3,Tata,2009,212000,Diesel,Individual,Manual,Third Owner,11.57,2179,138.1,7.0
4,Toyota,2010,125000,Diesel,Individual,Manual,Second Owner,11.5,2982,171,7.0
5,Maruti,2010,25000,Petrol,Individual,Manual,Second Owner,19.7,796,46.3,5.0


In [ ]:
# Create labels dataframe
label = ['selling_price']
y = df[label]
y.head()

,selling_price
1,459999
2,1100000
3,229999
4,800000
5,180000


In [ ]:
# Run label encoding for all categorical features
from sklearn.preprocessing import LabelEncoder
features = ['brand', 'fuel', 'seller_type', 'owner', 'transmission', 'year']

for feat in features:
  X[feat] = LabelEncoder().fit_transform(X[feat].astype(str))

X.head()

,brand,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats
1,20,18,87000,1,1,1,0,20.77,1248,88.76,7.0
2,3,16,102000,1,0,0,0,19.62,1995,187.74,5.0
3,26,15,212000,1,1,1,4,11.57,2179,138.1,7.0
4,27,16,125000,1,1,1,2,11.5,2982,171,7.0
5,20,16,25000,3,1,1,2,19.7,796,46.3,5.0


In [ ]:
# Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# Splitting
from sklearn.model_selection import train_test_split
test_size = 0.10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 0)

In [ ]:
y_train.describe()

,selling_price
count,4.344000e+03
mean,5.286893e+05
std,5.122169e+05
min,2.999900e+04
25%,2.500000e+05
50%,4.200000e+05
75%,6.490000e+05
max,7.200000e+06


In [ ]:
y_test.describe()

,selling_price
count,4.830000e+02
mean,5.105248e+05
std,4.668180e+05
min,4.500000e+04
25%,2.574995e+05
50%,4.200000e+05
75%,6.500000e+05
max,6.000000e+06


In [ ]:
# Create the model
import catboost as cb
model1 = cb.CatBoostRegressor(logging_level='Silent')

In [ ]:
# Train the model
model1.fit(X_train,y_train)
pred1 = model1.predict(X_test)

In [ ]:
# Show the model score for the test data
from sklearn.metrics import r2_score
r21 = r2_score(y_test, pred1)
print('R2: {:.5f}'.format(r21))

R2: 0.94258


### Experimento 2
* Objetivo: Verificar a performance do modelo, usando como parâmetros One-Hot-Encoding, StandardScaler e CatboostRegressor.
* Encoder: One-Hot-Encoding
* Scaler: StandardScaler
* Algoritm: CatBoostRegressor

In [ ]:
# Create features dataframe
features = ['brand', 'year', 'km_driven', 'fuel', 'seller_type', 'transmission', 'owner', 'mileage', 'engine', 'max_power', 'seats']
X = df[features]
X.head()

,brand,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats
1,Maruti,2012,87000,Diesel,Individual,Manual,First Owner,20.77,1248,88.76,7.0
2,BMW,2010,102000,Diesel,Dealer,Automatic,First Owner,19.62,1995,187.74,5.0
3,Tata,2009,212000,Diesel,Individual,Manual,Third Owner,11.57,2179,138.1,7.0
4,Toyota,2010,125000,Diesel,Individual,Manual,Second Owner,11.5,2982,171,7.0
5,Maruti,2010,25000,Petrol,Individual,Manual,Second Owner,19.7,796,46.3,5.0


In [ ]:
# Create labels dataframe
label = ['selling_price']
y = df[label]
y.head()

,selling_price
1,459999
2,1100000
3,229999
4,800000
5,180000


In [ ]:
# Run one-hot-encoding for all categorical features
X = pd.get_dummies(X, columns=['brand', 'fuel', 'seller_type', 'transmission', 'owner'])
X.head()

,year,km_driven,mileage,engine,max_power,seats,brand_Ambassador,brand_Ashok,brand_Audi,brand_BMW,brand_Chevrolet,brand_Daewoo,brand_Datsun,brand_Fiat,brand_Force,brand_Ford,brand_Honda,brand_Hyundai,brand_Isuzu,brand_Jaguar,brand_Jeep,brand_Kia,brand_Land,brand_Lexus,brand_MG,brand_Mahindra,brand_Maruti,brand_Mercedes-Benz,brand_Mitsubishi,brand_Nissan,brand_Renault,brand_Skoda,brand_Tata,brand_Toyota,brand_Volkswagen,brand_Volvo,fuel_CNG,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Automatic,transmission_Manual,owner_First Owner,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
1,2012,87000,20.77,1248,88.76,7.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0
2,2010,102000,19.62,1995,187.74,5.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0
3,2009,212000,11.57,2179,138.1,7.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1
4,2010,125000,11.5,2982,171,7.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0
5,2010,25000,19.7,796,46.3,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0


In [ ]:
# Scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# Splitting
test_size = 0.10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 0)

In [ ]:
y_train.describe()

,selling_price
count,4.344000e+03
mean,5.286893e+05
std,5.122169e+05
min,2.999900e+04
25%,2.500000e+05
50%,4.200000e+05
75%,6.490000e+05
max,7.200000e+06


In [ ]:
y_test.describe()

,selling_price
count,4.830000e+02
mean,5.105248e+05
std,4.668180e+05
min,4.500000e+04
25%,2.574995e+05
50%,4.200000e+05
75%,6.500000e+05
max,6.000000e+06


In [ ]:
# Create the model
model2 = cb.CatBoostRegressor(logging_level='Silent')

In [ ]:
# Train the model
model2.fit(X_train,y_train)
pred2 = model2.predict(X_test)

In [ ]:
# Show the model score for the test data
r22 = r2_score(y_test, pred2)
print('R2: {:.5f}'.format(r22))

R2: 0.94824


### Experimento 3
* Objetivo: Verificar a performance do modelo, usando como parâmetros One-Hot-Encoding + TargetEncoding, StandardScaler e CatboostRegressor.
* Encoder: One-Hot-Encoding + TargetEndcoding
* Scaler: StandardScaler
* Algoritm: CatBoostRegressor

In [ ]:
# Create features dataframe
features = ['brand', 'year', 'km_driven', 'fuel', 'seller_type', 'transmission', 'owner', 'mileage', 'engine', 'max_power', 'seats']
X = df[features]
X.head()

,brand,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats
1,Maruti,2012,87000,Diesel,Individual,Manual,First Owner,20.77,1248,88.76,7.0
2,BMW,2010,102000,Diesel,Dealer,Automatic,First Owner,19.62,1995,187.74,5.0
3,Tata,2009,212000,Diesel,Individual,Manual,Third Owner,11.57,2179,138.1,7.0
4,Toyota,2010,125000,Diesel,Individual,Manual,Second Owner,11.5,2982,171,7.0
5,Maruti,2010,25000,Petrol,Individual,Manual,Second Owner,19.7,796,46.3,5.0


In [ ]:
# Create labels dataframe
label = ['selling_price']
y = df[label]
y.head()

,selling_price
1,459999
2,1100000
3,229999
4,800000
5,180000


In [ ]:
# Run one-hot-encoding for all categorical features minus brand
X = pd.get_dummies(X, columns=['fuel', 'seller_type', 'transmission', 'owner'])

# Run target encoding for brand feature
from category_encoders import TargetEncoder
encoder = TargetEncoder()
X['brand'] = encoder.fit_transform(X['brand'], y)

X.head()

,brand,year,km_driven,mileage,engine,max_power,seats,fuel_CNG,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Automatic,transmission_Manual,owner_First Owner,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
1,3.916008e+05,2012,87000,20.77,1248,88.76,7.0,0,1,0,0,0,1,0,0,1,1,0,0,0,0
2,2.970455e+06,2010,102000,19.62,1995,187.74,5.0,0,1,0,0,1,0,0,1,0,1,0,0,0,0
3,3.488978e+05,2009,212000,11.57,2179,138.1,7.0,0,1,0,0,0,1,0,0,1,0,0,0,0,1
4,9.530330e+05,2010,125000,11.5,2982,171,7.0,0,1,0,0,0,1,0,0,1,0,0,1,0,0
5,3.916008e+05,2010,25000,19.7,796,46.3,5.0,0,0,0,1,0,1,0,0,1,0,0,1,0,0


In [ ]:
# Scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# Splitting
test_size = 0.10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 0)

In [ ]:
y_train.describe()

,selling_price
count,4.344000e+03
mean,5.286893e+05
std,5.122169e+05
min,2.999900e+04
25%,2.500000e+05
50%,4.200000e+05
75%,6.490000e+05
max,7.200000e+06


In [ ]:
y_test.describe()

,selling_price
count,4.830000e+02
mean,5.105248e+05
std,4.668180e+05
min,4.500000e+04
25%,2.574995e+05
50%,4.200000e+05
75%,6.500000e+05
max,6.000000e+06


In [ ]:
# Create the model
model3 = cb.CatBoostRegressor(logging_level='Silent')

In [ ]:
# Train the model
model3.fit(X_train,y_train)
pred3 = model3.predict(X_test)

In [ ]:
# Show the model score for the test data
r23 = r2_score(y_test, pred3)
print('R2: {:.5f}'.format(r23))

R2: 0.94563


### Experimento 4
* Objetivo: Verificar a performance dos 3 modelos obtidos nos experimentos 1, 2 e 3 com linhas duplicadas.
* Encoder: One-Hot-Encoding + TargetEndcoding
* Scaler: StandardScaler
* Algoritm: CatBoostRegressor

In [ ]:
# Restore df1 dataframe copy to df
df = df1

#### Modelo 1

In [ ]:
# Splitting
features = ['brand', 'year', 'km_driven', 'fuel', 'seller_type', 'transmission', 'owner', 'mileage', 'engine', 'max_power', 'seats']
X = df[features]
label = ['selling_price']
y = df[label]

# Preprocessing (LabelEncoding)
features = ['brand', 'fuel', 'seller_type', 'owner', 'transmission', 'year']
for feat in features:
  X[feat] = LabelEncoder().fit_transform(X[feat].astype(str))

scaler = StandardScaler()
X = scaler.fit_transform(X)
test_size = 0.10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 0)

# Prediction
model1c = cb.CatBoostRegressor(logging_level='Silent')
model1c.fit(X_train,y_train)
pred1c = model1c.predict(X_test)
r21c = r2_score(y_test, pred1c)
print('R2: {:.5f}'.format(r21c))

R2: 0.97921


#### Modelo 2

In [ ]:
# Splitting
features = ['brand', 'year', 'km_driven', 'fuel', 'seller_type', 'transmission', 'owner', 'mileage', 'engine', 'max_power', 'seats']
X = df[features]
label = ['selling_price']
y = df[label]

# Preprocessing (One-Hot-Encoding)
X = pd.get_dummies(X, columns=['brand', 'fuel', 'seller_type', 'transmission', 'owner'])
scaler = StandardScaler()
X = scaler.fit_transform(X)
test_size = 0.10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 0)

# Prediction
model2c = cb.CatBoostRegressor(logging_level='Silent')
model2c.fit(X_train,y_train)
pred2c = model2c.predict(X_test)
r22c = r2_score(y_test, pred2c)
print('R2: {:.5f}'.format(r22c))

R2: 0.97902


#### Modelo 3

In [ ]:
# Splitting
features = ['brand', 'year', 'km_driven', 'fuel', 'seller_type', 'transmission', 'owner', 'mileage', 'engine', 'max_power', 'seats']
X = df[features]
label = ['selling_price']
y = df[label]

# Preprocessing (One-Hot-Encoding + TargetEncoding)
X = pd.get_dummies(X, columns=['fuel', 'seller_type', 'transmission', 'owner'])
encoder = TargetEncoder()
X['brand'] = encoder.fit_transform(X['brand'], y)
scaler = StandardScaler()
X = scaler.fit_transform(X)
test_size = 0.10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 0)

# Prediction
model3c = cb.CatBoostRegressor(logging_level='Silent')
model3c.fit(X_train,y_train)
pred3c = model3c.predict(X_test)
r23c = r2_score(y_test, pred3c)
print('R2: {:.5f}'.format(r23c))

R2: 0.98236


## Resultados
A seguir estão os valores do R2 Score para cada um dos quatro experimentos:

In [ ]:
# Show result table
pd.DataFrame({'Modelo': ['1', '2', '3'], 'R2(s/dup.)': [r21, r22, r23], 'R2(c/dup.)': [r21c, r22c, r23c]})

,Modelo,R2(s/dup.),R2(c/dup.)
0,1,0.942583,0.979211
1,2,0.948240,0.979022
2,3,0.945627,0.982362


Avaliando os valores do R2 score dos quatro experimentos, é possível verificar que o modelo que se saiu melhor foi o modelo 3 com dados duplicados. Desta forma, vamos utilizar o modelo 3, com dados duplicados, para a predição dos dados de validação.

# Validação

In [ ]:
# Load validation dataset
df_val = pd.read_csv('https://raw.githubusercontent.com/xmrcl0/SME0829/main/project2/data/test_car_details.csv')
df_val.head()

,Id,name,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,1,Tata Indigo LS,2008,60000,Diesel,Individual,Manual,Third Owner,17.0 kmpl,1405 CC,70 bhp,"13.5@ 2,500(kgm@ rpm)",5.0
1,2,Maruti Baleno Delta 1.2,2019,30000,Petrol,Individual,Manual,First Owner,21.4 kmpl,1197 CC,83.1 bhp,115Nm@ 4000rpm,5.0
2,3,Maruti Swift Dzire AMT ZDI,2017,80000,Diesel,Individual,Automatic,First Owner,28.4 kmpl,1248 CC,74.02 bhp,190Nm@ 2000rpm,5.0
3,4,Mahindra XUV500 W10 2WD,2018,30000,Diesel,Individual,Manual,First Owner,16.0 kmpl,2179 CC,140 bhp,330Nm@ 1600-2800rpm,7.0
4,5,Hyundai Elite i20 Asta Option BSIV,2019,35000,Petrol,Individual,Manual,First Owner,18.6 kmpl,1197 CC,81.86 bhp,117nm@ 4000rpm,5.0


In [ ]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2375 entries, 0 to 2374
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            2375 non-null   int64  
 1   name          2375 non-null   object 
 2   year          2375 non-null   int64  
 3   km_driven     2375 non-null   int64  
 4   fuel          2375 non-null   object 
 5   seller_type   2375 non-null   object 
 6   transmission  2375 non-null   object 
 7   owner         2375 non-null   object 
 8   mileage       2375 non-null   object 
 9   engine        2375 non-null   object 
 10  max_power     2375 non-null   object 
 11  torque        2375 non-null   object 
 12  seats         2375 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 241.3+ KB


### Preparação dos dados
Dado que o modelo escolhido foi o modelo 3, devemos preparar o dataset de validação com os mesmos parâmetros utilizados do experimento 3 (Encoding: One-Hot-Encoding + Target Encoding e Scaler: StandardScaler).

In [ ]:
# Remove Id column
X_val = df_val.drop(labels='Id', axis=1)

# Replace car's name column by the car's brand
X_val['name'] = X_val['name'].apply(lambda name: pd.Series(name.split()[0]))
X_val.rename(columns={'name':'brand'}, inplace=True)

# Remove unit from the max_power feature
X_val['max_power'] = X_val['max_power'].apply(lambda power: pd.Series(power.split()[0]))

# Remove unit from the engine feature
X_val['engine'] = X_val['engine'].apply(lambda power: pd.Series(power.split()[0]))

# Remove unit from the mileage feature
X_val['mileage'] = X_val['mileage'].apply(lambda power: pd.Series(power.split()[0]))

# Remove rows with empty torque and after the entire column (it helps, believe)
X_val = X_val.drop(labels='torque', axis=1)

# Run One-Hot-Encoding for all categorical columns
X_val = pd.get_dummies(X_val, columns=['fuel', 'seller_type', 'transmission', 'owner'])

# Run Target Encoding
X_val['brand'] = encoder.transform(X_val['brand'])

# Scaling
X_val = scaler.fit_transform(X_val)

### Predições

In [ ]:
# Run the prediction routine for the validation dataset using the best model (model3)
y_predic = model3c.predict(X_val)
y_predic[:, None]

array([[ 102294.41135241],
       [ 605353.95564056],
       [ 655760.0316992 ],
       ...,
       [ 750800.16869921],
       [2152107.40749485],
       [ 671287.47127336]])

In [ ]:
# Show some predictions see if it makes sense
pd.DataFrame({'Id': df_val['Id'], 'name': df_val['name'], 'year': df_val['year'],'selling_price': y_predic})

,Id,name,year,selling_price
0,1,Tata Indigo LS,2008,1.022944e+05
1,2,Maruti Baleno Delta 1.2,2019,6.053540e+05
2,3,Maruti Swift Dzire AMT ZDI,2017,6.557600e+05
3,4,Mahindra XUV500 W10 2WD,2018,1.364413e+06
4,5,Hyundai Elite i20 Asta Option BSIV,2019,5.723771e+05
...,...,...,...,...
2370,2371,Volkswagen Vento Diesel Highline,2011,3.419358e+05
2371,2372,Hyundai Creta 1.6 CRDi SX Plus,2016,8.594161e+05
2372,2373,Ford EcoSport 1.5 Diesel Titanium Plus BSIV,2017,7.508002e+05
2373,2374,Toyota Fortuner 4x4 AT,2016,2.152107e+06


In [ ]:
# Save preditions to csv
submission_data = pd.DataFrame({'Id': df_val['Id'], 'selling_price': y_predic})
pd.DataFrame(submission_data).to_csv("submission.csv", index=False)
submission_data

,Id,selling_price
0,1,1.022944e+05
1,2,6.053540e+05
2,3,6.557600e+05
3,4,1.364413e+06
4,5,5.723771e+05
...,...,...
2370,2371,3.419358e+05
2371,2372,8.594161e+05
2372,2373,7.508002e+05
2373,2374,2.152107e+06


# Conclusão
Neste projeto, construímos um modelo estimador de preços de venda para carros usados.

Ao todo foram realizados três experimentos, com o objetivo de verificar a influência do tipo de encoder nas variáveis categóricas, e um experimento com o objetivo de verificar a influência do uso de linhas duplicadas na qualidade do modelo. Os quatro experimentos resultaram em seis valores de r2 score, através dos quais foi possível selecionar o modelo (modelo 3 com linhas duplicadas) que obteve o melhor comportamento preditivo (r2 score = **0.982362**), para os dados de treinamento (90% do dataset).

É interessante observar que o valor de 0.982362 foi obtido sem a *feature* "torque", a qual, a depender da capacidade preditiva do modelo, para os dados de validação, pode ser utilizada para tentar melhorar a qualidade do modelo. Além isso, outro ponto que vale a pena testar é a otimização dos hiperparametros do algoritmo CatBoostRegressor, uma vez que todos os experimentos foram realizados com os valores default.